In [51]:
'''Importing necessary libraries'''
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from markdown import  *
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# access the web content
headers = {"User-Agent": "Mozilla/5.0"}

In [53]:
loader = PyPDFLoader("https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf",headers=headers)

In [54]:
docs = loader.load()

In [55]:
docs[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-10-14T17:09:55+05:30', 'moddate': '2024-10-14T18:54:19+05:30', 'source': 'https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='9th Floor Nirmal Building Nariman Point Mumbai 400 021 \nTel 91 22 6778 9595 Fax 91 22 6630 3672 e-mail corporate.office@tcs.com website www.tcs.com \nRegistered Office 9th Floor Nirmal Building Nariman Point Mumbai 400 021 \nCorporate Identity No. (CIN): L22210MH1995PLC084781 \nTCS/SE/163/2024-25 \nOctober 14, 2024 \nNational Stock Exchange of India Limited BSE Limited \nExchange Plaza, C-1, Block G, P. J. Towers,   \nBandra Kurla Complex, Bandra (East)   Dalal Street, \nMumbai - 400051 Mumbai - 400001   \nSymbol - TCS Scrip Code No. 532540 \nDear Sirs, \n Sub: Transcript of the earnings conference call for the quarte

In [56]:
# for doc in docs:
#     print(type(doc.page_content)) # str
#     print(type(doc.metadata)) #  dict

In [57]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
)

In [58]:
chunks = text_splitter.split_documents(docs)

In [59]:
print(f"Split into {len(chunks)} chunks.")

Split into 79 chunks.


In [60]:
print(chunks[0])

page_content='9th Floor Nirmal Building Nariman Point Mumbai 400 021 
Tel 91 22 6778 9595 Fax 91 22 6630 3672 e-mail corporate.office@tcs.com website www.tcs.com 
Registered Office 9th Floor Nirmal Building Nariman Point Mumbai 400 021 
Corporate Identity No. (CIN): L22210MH1995PLC084781 
TCS/SE/163/2024-25 
October 14, 2024 
National Stock Exchange of India Limited BSE Limited 
Exchange Plaza, C-1, Block G, P. J. Towers,   
Bandra Kurla Complex, Bandra (East)   Dalal Street, 
Mumbai - 400051 Mumbai - 400001   
Symbol - TCS Scrip Code No. 532540 
Dear Sirs, 
 Sub: Transcript of the earnings conference call for the quarter and six-month period
ended September 30, 2024 
Pursuant to Regulation 30 of the SEBI (Listing Obligations and Disclosure Requirements) 
Regulations, 2015, please find enclosed the transcript of the earnings conference call for the 
quarter and six-month period ended September 30, 2024, conducted after the meeting of' metadata={'producer': 'Microsoft® Word for Microsof

In [61]:
print(chunks[1])

page_content='Regulations, 2015, please find enclosed the transcript of the earnings conference call for the 
quarter and six-month period ended September 30, 2024, conducted after the meeting of 
Board of Directors held on October 10, 2024, for your information and records. 
The above information is also available on the website of the Company: www.tcs.com. 
Thanking you, 
Yours faithfully, 
For Tata Consultancy Services Limited  
Pradeep Manohar Gaitonde 
Company Secretary 
Encl: As above' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-10-14T17:09:55+05:30', 'moddate': '2024-10-14T18:54:19+05:30', 'source': 'https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}


In [62]:
chunks

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-10-14T17:09:55+05:30', 'moddate': '2024-10-14T18:54:19+05:30', 'source': 'https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='9th Floor Nirmal Building Nariman Point Mumbai 400 021 \nTel 91 22 6778 9595 Fax 91 22 6630 3672 e-mail corporate.office@tcs.com website www.tcs.com \nRegistered Office 9th Floor Nirmal Building Nariman Point Mumbai 400 021 \nCorporate Identity No. (CIN): L22210MH1995PLC084781 \nTCS/SE/163/2024-25 \nOctober 14, 2024 \nNational Stock Exchange of India Limited BSE Limited \nExchange Plaza, C-1, Block G, P. J. Towers,   \nBandra Kurla Complex, Bandra (East)   Dalal Street, \nMumbai - 400051 Mumbai - 400001   \nSymbol - TCS Scrip Code No. 532540 \nDear Sirs, \n Sub: Transcript of the earnings conference call for the quart

In [63]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk}")  

Chunk 0: page_content='9th Floor Nirmal Building Nariman Point Mumbai 400 021 
Tel 91 22 6778 9595 Fax 91 22 6630 3672 e-mail corporate.office@tcs.com website www.tcs.com 
Registered Office 9th Floor Nirmal Building Nariman Point Mumbai 400 021 
Corporate Identity No. (CIN): L22210MH1995PLC084781 
TCS/SE/163/2024-25 
October 14, 2024 
National Stock Exchange of India Limited BSE Limited 
Exchange Plaza, C-1, Block G, P. J. Towers,   
Bandra Kurla Complex, Bandra (East)   Dalal Street, 
Mumbai - 400051 Mumbai - 400001   
Symbol - TCS Scrip Code No. 532540 
Dear Sirs, 
 Sub: Transcript of the earnings conference call for the quarter and six-month period
ended September 30, 2024 
Pursuant to Regulation 30 of the SEBI (Listing Obligations and Disclosure Requirements) 
Regulations, 2015, please find enclosed the transcript of the earnings conference call for the 
quarter and six-month period ended September 30, 2024, conducted after the meeting of' metadata={'producer': 'Microsoft® Word for

In [65]:
def create_vector_db(chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    vector_db = Chroma.from_documents(chunks, embeddings)
    return vector_db

In [66]:
retriever = create_vector_db(chunks).as_retriever(
    search_type="similarity"
)

### 🤖 LLM Integration Part

In [67]:
llm = GoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3, max_output_tokens=4096)

In [48]:
# response = llm.invoke("Who is TCS")
# response

In [68]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [69]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [49]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [70]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [71]:
user_input = input("Enter Query Based on DOCS: ")

In [72]:
results = rag_chain.invoke({"input": user_input})

In [78]:
pprint(results)

{'answer': 'The transcript of the earnings conference call for the quarter and '
           'six-month period ending September 30, 2024, is available.  The '
           'call was conducted after the Board of Directors meeting on October '
           '10, 2024, and discusses Tata Consultancy Services (TCS) earnings.  '
           'It is available at www.tcs.com.',
 'context': [Document(metadata={'creationdate': '2024-10-14T17:09:55+05:30', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2024-10-14T18:54:19+05:30', 'page': 0, 'page_label': '1', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf', 'total_pages': 28}, page_content='Regulations, 2015, please find enclosed the transcript of the earnings conference call for the \nquarter and six-month period ended September 30, 2024, conducted after the meeting of \nBoard of Directors held on October 10, 2024, for your informat

In [79]:
from pprint import  pprint
pprint(results["answer"])

('The transcript of the earnings conference call for the quarter and six-month '
 'period ending September 30, 2024, is available.  The call was conducted '
 'after the Board of Directors meeting on October 10, 2024, and discusses Tata '
 'Consultancy Services (TCS) earnings.  It is available at www.tcs.com.')


In [77]:
print(results["context"][0].metadata)

{'creationdate': '2024-10-14T17:09:55+05:30', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2024-10-14T18:54:19+05:30', 'page': 0, 'page_label': '1', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'https://www.bseindia.com/xml-data/corpfiling/AttachHis/a7066eb7-d5d8-4d9e-87fd-ed222315999d.pdf', 'total_pages': 28}
